In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import friedmanchisquare, wilcoxon, mannwhitneyu
from mapper import continent_mapper, fullname_to_short, politicians_db
from itertools import combinations

import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

In [13]:
def is_nan_or_list_with_nan(obj):
    try:
        if any(pd.isna(elem) for elem in obj):
            return True
    except TypeError:
        if pd.isna(obj):
            return True
    return False

In [14]:
class ComprehensivePoliticalAnalyzer:
    def __init__(self, phase2_files):
        """
        Initialize the analyzer with both phase datasets
        """
        self.continent_mapping = continent_mapper
        self.politician_info = politicians_db
        self.phase2_df = self.load_and_clean_data(phase2_files, phase=2)
    
    def load_and_clean_data(self, filepath, phase):
        """
        Load and clean the data, with politician identification
        """
        dfs = []  # list to collect all DataFrames

        for path in filepath:
            df = pd.read_csv(path)

            # Extract politician short name from file path, e.g., "phase1_data/albanese_phase1_data.csv"
            politician_name = path.split("/")[1].split('_')[0].lower()
            df['politician_id'] = politician_name

            if phase == 2:
                # Clean phase 2 ranking columns
                ranking_cols = [
                    'competence', 'likable', 'trustworthy', 'approachable', 
                    'charismatic', 'aggressive', 'friendly', 'professional', 
                    'support', 'positive', 'visually_appealing', 'overall_attention'
                ]
                for col in ranking_cols:
                    if col in df.columns:
                        df[col] = df[col].apply(self.parse_ranking)

            # Add continent based on country
            df['continent'] = df['country'].map(self.continent_mapping)
            df['phase'] = phase

            # Add politician attributes
            for idx, row in df.iterrows():
                pid = row.get('politician_id', 'unknown')
                if pid in self.politician_info:
                    pdata = self.politician_info[pid]
                    df.loc[idx, 'politician_name'] = pdata['name']
                    df.loc[idx, 'politician_country'] = pdata['country']
                    df.loc[idx, 'politician_continent'] = pdata['continent']
                    df.loc[idx, 'ideology'] = pdata['ideology']
                    df.loc[idx, 'popularity_score'] = pdata['popularity_score']
                    df.loc[idx, 'incumbent_status'] = pdata['incumbent_status']
                    df.loc[idx, 'same_country'] = row['country'] == pdata['country']
                    df.loc[idx, 'same_continent'] = df.loc[idx, 'continent'] == pdata['continent']

            dfs.append(df)

        # Combine all individual DataFrames into one
        combined_df = pd.concat(dfs, ignore_index=True)
        return combined_df
    
    def parse_ranking(self, ranking_str):
        """Parse ranking string and return list of ranks"""
        if pd.isna(ranking_str) or ranking_str == '':
            return np.nan
        
        try:
            ranks = [int(x.strip()) for x in str(ranking_str).split(',') if x.strip()]
            return ranks
        except:
            return np.nan
    
    def extract_phase2_rankings_by_politician(self, trait_cols):
        """
        Extract rankings from Phase 2 data, organized by politician
        """
        results = []
        
        for idx, row in self.phase2_df.iterrows():
            for trait in trait_cols:
                if trait in self.phase2_df.columns and not is_nan_or_list_with_nan(row[trait]):
                    ranks = row[trait]
                    if isinstance(ranks, list) and len(ranks) == 4:
                        result_row = {
                            'response_id': row['response_id'],
                            'politician_id': row.get('politician_id', 'unknown'),
                            'politician_name': row.get('politician_name', 'Unknown'),
                            'age': row['age'],
                            'sex': row['sex'],
                            'participant_country': row['country'],
                            'participant_continent': row['continent'],
                            'politician_country': row.get('politician_country', 'Unknown'),
                            'politician_continent': row.get('politician_continent', 'Unknown'),
                            'same_country': row.get('same_country', False),
                            'same_continent': row.get('same_continent', False),
                            'ideology': row.get('ideology', 'unknown'),
                            'popularity_score': row.get('popularity_score', 0),
                            'political_engagement': row['political_engagement'],
                            'political_alignment': row['political_alignment'],
                            'familiarity': row['familiarity'],
                            'trait': trait,
                            'original_rank': ranks[0],
                            'background_change_rank': ranks[1],
                            'text_change_rank': ranks[2],
                            'font_change_rank': ranks[3]
                        }
                        results.append(result_row)
        
        return pd.DataFrame(results)
    
    def analyze_by_politician_strategy(self, strategy='pooled'):
        """
        Choose analysis strategy: 'pooled', 'individual', or 'comparative'
        """
        key_traits = ["competence", "likable", "trustworthy", "approachable", "charismatic",
                     "aggressive", "friendly", "professional", "support", "positive",
                     "visually_appealing", "overall_attention"]

        ranking_df = self.extract_phase2_rankings_by_politician(key_traits)
        print(f"Number of unique participants: {len(ranking_df['response_id'].unique())}")
        
        if ranking_df.empty:
            print("No valid ranking data found")
            return
        
        print(f"="*80)
        print(f"ANALYSIS STRATEGY: {strategy.upper()}")
        print(f"="*80)
        
        if strategy == 'pooled':
            self._analyze_pooled_data(ranking_df, key_traits)
        elif strategy == 'comparative':
            self._analyze_comparative_politicians(ranking_df, key_traits)
        else:
            print(f"Unknown strategy: {strategy}")
    
    def _analyze_pooled_data(self, ranking_df, key_traits):
        """
        Analyze all politicians together (main effects)
        """
        print("POOLED ANALYSIS - All Politicians Combined")
        print("-" * 50)
        
        for trait in key_traits:
            trait_data = ranking_df[ranking_df['trait'] == trait]
            if trait_data.empty:
                continue
            
            print(f"\n{trait.upper()} Analysis:")
            
            # Main effects (all politicians pooled)
            conditions = ['original_rank', 'background_change_rank', 'text_change_rank', 'font_change_rank']
            condition_names = ['Original', 'Background Change', 'Text Change', 'Font Change']
            
            ranks_by_condition = {}
            for condition in conditions:
                ranks_by_condition[condition] = trait_data[condition].dropna()
            
            # Friedman test for overall differences
            valid_ranks = [ranks for ranks in ranks_by_condition.values() if len(ranks) > 0]
            if len(valid_ranks) >= 3:
                try:
                    stat, p_val = friedmanchisquare(*valid_ranks)
                    print(f"Friedman test: χ² = {stat:.3f}, p = {p_val:.3f}")
                    
                    if p_val < 0.05:
                        print("Significant overall differences detected")
                        # Post-hoc pairwise comparisons
                        self._pairwise_comparisons(trait_data, conditions)
                    else:
                        print("No significant overall differences")
                except Exception as e:
                    print(f"Error in Friedman test: {e}")
            
            # Show descriptive statistics
            print("\nDescriptive Statistics:")
            for condition, name in zip(conditions, condition_names):
                ranks = trait_data[condition].dropna()
                if len(ranks) > 0:
                    print(f"{name}: M = {ranks.mean():.2f}, SD = {ranks.std():.2f}, n = {len(ranks)}")
    
    def _analyze_comparative_politicians(self, ranking_df, key_traits):
        """Compare effects across politicians (background, text, font) for key traits."""
        print("COMPARATIVE POLITICIAN ANALYSIS")
        print("-" * 50)

        politicians = [p for p in ranking_df['politician_name'].unique() if not pd.isna(p)]
        if len(politicians) < 2:
            print("Need at least 2 politicians for comparative analysis")
            return

        print(f"Comparing {len(politicians)} politicians: {', '.join(politicians)}")

        # Calculate effect sizes for each politician
        politician_effects = {}
        manipulations = [
            ('bg_effect', 'background_change_rank', 'Background Color'),
            ('text_effect', 'text_change_rank', 'Text Color'),
            ('font_effect', 'font_change_rank', 'Font Style')
        ]
        
        focus_trait = [
            "competence", "likable", "trustworthy", "approachable", "charismatic",
            "aggressive", "friendly", "professional", "support", "positive",
            "visually_appealing", "overall_attention"
        ]

        for politician in politicians:
            pol_data = ranking_df[ranking_df['politician_name'] == politician]
            sample_row = pol_data.iloc[0]
            effects = {
                'popularity': sample_row.get('popularity_score', 0),
                'ideology': sample_row.get('ideology', 'unknown'),
                'country': sample_row.get('politician_country', 'Unknown')
            }
            for trait in key_traits:
                trait_data = pol_data[pol_data['trait'] == trait]
                original = trait_data['original_rank'].dropna()
                for effect_key, changed_col, _ in manipulations:
                    changed = trait_data[changed_col].dropna()
                    if len(original) >= 5 and len(changed) >= 5:
                        effects[f'{trait}_{effect_key}'] = original.mean() - changed.mean()
            politician_effects[politician] = effects

        # Print summary tables for each manipulation
        for effect_key, _, label in manipulations:
            print(f"\n{label} Effects by Politician:")
            print(f"{'Politician':<20} {'Popularity':<10} {'Ideology':<14} " +
                " ".join([f"{t.capitalize():<12}" for t in focus_trait]))
            print("-" * 80)
            for politician in politicians:
                effects = politician_effects[politician]
                popularity = effects.get('popularity', 0)
                ideology = str(effects.get('ideology', 'unknown'))[:14]
                trait_vals = [effects.get(f'{trait}_{effect_key}', 0) for trait in focus_trait]
                print(f"{fullname_to_short.get(politician, politician):<20} {popularity:<10} {ideology:<14} " +
                    " ".join([f"{v:>11.2f}" for v in trait_vals]))
                
                
        self._analyze_popularity_correlations(politician_effects, key_traits)
    
    def _analyze_popularity_correlations(self, politician_effects, key_traits):
        """
        Analyze correlations between politician popularity and visual effects
        for background, text, and font manipulations.

        Threshold for interpreting correlation strength:
            0.00–0.10   Negligible
            0.10–0.39   Weak
            0.40–0.69   Moderate
            0.70–0.89   Strong
            0.90–1.00   Very strong
        """
        from scipy import stats

        correlation_thresholds = [
            (0.00, 0.10, "Negligible"),
            (0.10, 0.39, "Weak"),
            (0.40, 0.69, "Moderate"),
            (0.70, 0.89, "Strong"),
            (0.90, 1.00, "Very strong")
        ]

        manipulations = [
            ('bg_effect', 'Background Color'),
            ('text_effect', 'Text Color'),
            ('font_effect', 'Font Style')
        ]

        def interpret_strength(r):
            r_abs = abs(r)
            for low, high, label in correlation_thresholds:
                if low <= r_abs <= high:
                    return label
            return "Unknown"

        print(f"\nPopularity-Effect Correlations:")
        print("=" * 80)

        for trait in key_traits:
            print(f"\nTrait: {trait.capitalize()}")
            print("-" * 40)

            for effect_key, label in manipulations:
                popularity_scores = []
                manipulation_effects = []

                for politician, effects in politician_effects.items():
                    col_name = f"{trait}_{effect_key}"
                    if col_name in effects and 'popularity' in effects:
                        popularity_scores.append(effects['popularity'])
                        manipulation_effects.append(effects[col_name])

                if len(popularity_scores) < 3:
                    print(f"  {label}: Not enough data (n={len(popularity_scores)})")
                    continue

                try:
                    correlation, p_val = stats.pearsonr(popularity_scores, manipulation_effects)
                except Exception as e:
                    print(f"  {label}: Correlation calculation failed: {e}")
                    continue

                strength = interpret_strength(correlation)
                direction = ("more popular politicians benefit more"
                            if correlation > 0 else
                            "less popular politicians benefit more")

                print(f"  {label}: r = {correlation:+.3f}, p = {p_val:.3f} → {strength} correlation")
                if abs(correlation) >= 0.40 and p_val < 0.10:
                    print(f"    **{strength} correlation:** {direction} from original design")

    
    
    def analyze_geographical_effects(self):
        """
        Enhanced geographical analysis for ranking data.
        Compares manipulation effects (background, text, font) for same vs different continent participant-politician pairs.

        Assumes: Lower rank number = better rating (e.g., 1 = best, 4 = worst).
        Positive effect value = improvement after manipulation.
        """

        key_traits = [
            "competence", "likable", "trustworthy", "approachable", "charismatic",
            "aggressive", "friendly", "professional", "support", "positive",
            "visually_appealing", "overall_attention"
        ]
        manipulations = [
            ('bg_effect', 'background_change_rank', 'Background Color'),
            ('text_effect', 'text_change_rank', 'Text Color'),
            ('font_effect', 'font_change_rank', 'Font Style')
        ]

        ranking_df = self.extract_phase2_rankings_by_politician(key_traits)

        if ranking_df.empty:
            print("No ranking data available.")
            return

        # Check data availability by continent
        participant_continents = ranking_df['participant_continent'].value_counts()
        politician_continents = ranking_df['politician_continent'].value_counts()

        print(f"Participant continents: {dict(participant_continents)}")
        print(f"Politician continents: {dict(politician_continents)}")

        # Only keep continents with enough data
        valid_continents = participant_continents[participant_continents >= 3].index
        if len(valid_continents) < 2:
            print("Insufficient data for continental comparison")
            return

        print(f"Analyzing continents with sufficient data: {list(valid_continents)}")

        # Detect whether low numbers = better ranking
        rank_min, rank_max = ranking_df['original_rank'].min(), ranking_df['original_rank'].max()
        low_is_better = rank_min < rank_max  # True if 1 is best

        # Main analysis
        for trait in key_traits:
            print(f"\n{'-'*60}")
            print(f"TRAIT: {trait.upper()}")
            print(f"{'-'*60}")

            trait_data = ranking_df[ranking_df['trait'] == trait].copy()

            # Filter continents
            trait_data = trait_data[
                trait_data['participant_continent'].isin(valid_continents) &
                trait_data['politician_continent'].isin(valid_continents)
            ]

            if len(trait_data) < 10:
                print(f"Insufficient data for {trait} analysis (n={len(trait_data)})")
                continue

            same_continent_data = trait_data[trait_data['same_continent']]
            diff_continent_data = trait_data[~trait_data['same_continent']]

            print(f"Same continent pairs: n={len(same_continent_data)}")
            print(f"Different continent pairs: n={len(diff_continent_data)}")

            if len(same_continent_data) < 5 or len(diff_continent_data) < 5:
                print("Insufficient data for same/different continent comparison")
                continue

            for _, changed_col, label in manipulations:
                print(f"\n  {label} Manipulation Effects:")

                if changed_col not in trait_data.columns:
                    print(f"    Missing column: {changed_col}")
                    continue

                # Calculate improvement: positive = better outcome
                if low_is_better:
                    calc_effect = lambda df: df['original_rank'] - df[changed_col]
                else:
                    calc_effect = lambda df: df[changed_col] - df['original_rank']

                same_effects = calc_effect(same_continent_data).dropna()
                diff_effects = calc_effect(diff_continent_data).dropna()

                if len(same_effects) < 3 or len(diff_effects) < 3:
                    print(f"    Insufficient valid data for {label} comparison")
                    continue

                # Summary
                same_mean, diff_mean = same_effects.mean(), diff_effects.mean()
                same_std, diff_std = same_effects.std(), diff_effects.std()

                def describe_effect(mean_val):
                    if abs(mean_val) < 0.1:
                        return "negligible"
                    elif abs(mean_val) <= 0.5:
                        return "small improvement" if mean_val > 0 else "small decline"
                    else:
                        return "meaningful improvement" if mean_val > 0 else "meaningful decline"

                print(f"    Same continent: Mean = {same_mean:+.3f} ± {same_std:.3f} (n={len(same_effects)}), {describe_effect(same_mean)}")
                print(f"    Diff continent: Mean = {diff_mean:+.3f} ± {diff_std:.3f} (n={len(diff_effects)}), {describe_effect(diff_mean)}")

                # Statistical comparison
                try:
                    stat, p_val = mannwhitneyu(same_effects, diff_effects, alternative='two-sided')
                    print(f"    Mann-Whitney U: U = {stat:.1f}, p = {p_val:.3f}")
                    if p_val < 0.05:
                        stronger = "same continent" if abs(same_mean) > abs(diff_mean) else "different continent"
                        print(f"    **Significant difference: {stronger} pairs show stronger effect**")
                except Exception as e:
                    print(f"    Statistical test failed: {e}")

            # Detailed breakdown
            print(f"\n  Detailed Breakdown by Continent Pair:")
            for participant_cont in valid_continents:
                p_data = trait_data[trait_data['participant_continent'] == participant_cont]
                if len(p_data) < 5:
                    continue
                print(f"    {participant_cont} participants (n={len(p_data)}):")
                for politician_cont in valid_continents:
                    pair_data = p_data[p_data['politician_continent'] == politician_cont]
                    if len(pair_data) < 3:
                        continue
                    rel_type = "SAME CONTINENT" if participant_cont == politician_cont else "DIFFERENT CONTINENT"
                    print(f"      → Rating {politician_cont} politicians (n={len(pair_data)}, {rel_type}):")
                    for _, changed_col, label in manipulations:
                        if changed_col in pair_data.columns:
                            effects = calc_effect(pair_data).dropna()
                            if len(effects) >= 3:
                                print(f"        {label}: {effects.mean():+.2f} (n={len(effects)})")

    def _pairwise_comparisons(self, trait_data, conditions):
        """Perform pairwise comparisons between conditions"""
        
        print("Pairwise comparisons:")
        condition_names = ['Original', 'Background', 'Text', 'Font']
        
        for i, (cond1, cond2) in enumerate(combinations(range(len(conditions)), 2)):
            ranks1 = trait_data[conditions[cond1]].dropna()
            ranks2 = trait_data[conditions[cond2]].dropna()
            
            if len(ranks1) >= 5 and len(ranks2) >= 5:
                try:
                    stat, p_val = wilcoxon(ranks1, ranks2, alternative='two-sided')
                    effect = ranks1.mean() - ranks2.mean()
                    print(f"  {condition_names[cond1]} vs {condition_names[cond2]}: p = {p_val:.3f}, effect = {effect:.3f}")
                except:
                    pass
                
    def visualize_trait_effects(self, trait='trustworthy'):
        """
        Visualize mean ± SE of ranks for each manipulation for a given trait (pooled across politicians).
        """

        key_traits = [
            "competence", "likable", "trustworthy", "approachable", "charismatic",
            "aggressive", "friendly", "professional", "support", "positive",
            "visually_appealing", "overall_attention"
        ]
        if trait not in key_traits:
            print(f"Trait '{trait}' not found.")
            return

        ranking_df = self.extract_phase2_rankings_by_politician(key_traits)
        trait_data = ranking_df[ranking_df['trait'] == trait]

        conditions = ['original_rank', 'background_change_rank', 'text_change_rank', 'font_change_rank']
        condition_names = ['Original', 'Background', 'Text', 'Font']

        means = []
        ses = []
        ns = []
        for cond in conditions:
            vals = trait_data[cond].dropna()
            means.append(vals.mean())
            ses.append(vals.std() / (len(vals)**0.5) if len(vals) > 0 else 0)
            ns.append(len(vals))

        plt.figure(figsize=(7, 5))
        sns.barplot(x=condition_names, y=means, yerr=ses, palette="husl", capsize=0.2)
        plt.ylabel("Mean Rank (Lower = Better)")
        plt.title(f"Trait: {trait.capitalize()} (n={min(ns)})")
        plt.tight_layout()
        plt.show()
    
    def run_recommended_analysis_pipeline(self):
        """
        Run the recommended analysis pipeline
        """
        print("COMPREHENSIVE POLITICAL PERCEPTION ANALYSIS")
        print("="*80)
        
        # Step 1: Pooled analysis for main effects
        print("\nSTEP 1: MAIN EFFECTS ANALYSIS")
        self.analyze_by_politician_strategy('pooled')
        
        # Step 2: Comparative analysis
        print(f"\n{'='*80}")
        print("STEP 2: COMPARATIVE ANALYSIS")
        self.analyze_by_politician_strategy('comparative')
        
        print(f"\n{'='*80}")
        print("STEP 3: GEOGRAPHICAL ANALYSIS")
        self.analyze_geographical_effects()
    


In [15]:

phase2_data = [
    'phase2_data/luxon_phase2_data.csv',
    'phase2_data/biden_phase2_data.csv',
    'phase2_data/prayut_phase2_data.csv',
    'phase2_data/trump_phase2_data.csv',
    'phase2_data/friedrich_phase2_data.csv',
    'phase2_data/sunak_phase2_data.csv',
    'phase2_data/boko_phase2_data.csv',
    'phase2_data/morrison_phase2_data.csv',
    'phase2_data/olaf_phase2_data.csv',
    'phase2_data/singh_phase2_data.csv',
    'phase2_data/paetongtarn_phase2_data.csv',
    'phase2_data/bolsonaro_phase2_data.csv',
    'phase2_data/trudeau_phase2_data.csv',
    'phase2_data/starmer_phase2_data.csv',
    'phase2_data/albanese_phase2_data.csv',
    'phase2_data/modi_phase2_data.csv',
    'phase2_data/lee_phase2_data.csv',
    'phase2_data/ruto_phase2_data.csv',
    'phase2_data/yoon_phase2_data.csv',
    'phase2_data/kenyatta_phase2_data.csv',
    'phase2_data/jacinda_phase2_data.csv',
    'phase2_data/masisi_phase2_data.csv',
    'phase2_data/lula_phase2_data.csv',
    'phase2_data/harper_phase2_data.csv',
]


analyzer = ComprehensivePoliticalAnalyzer(phase2_data)

# Run the recommended analysis pipeline
analyzer.run_recommended_analysis_pipeline()

COMPREHENSIVE POLITICAL PERCEPTION ANALYSIS

STEP 1: MAIN EFFECTS ANALYSIS
Number of unique participants: 124
ANALYSIS STRATEGY: POOLED
POOLED ANALYSIS - All Politicians Combined
--------------------------------------------------

COMPETENCE Analysis:
Friedman test: χ² = 226.798, p = 0.000
Significant overall differences detected
Pairwise comparisons:
  Original vs Background: p = 0.000, effect = -0.835
  Original vs Text: p = 0.000, effect = -1.221
  Original vs Font: p = 0.000, effect = -1.607
  Background vs Text: p = 0.000, effect = -0.386
  Background vs Font: p = 0.000, effect = -0.772
  Text vs Font: p = 0.000, effect = -0.386

Descriptive Statistics:
Original: M = 1.58, SD = 0.86, n = 267
Background Change: M = 2.42, SD = 1.04, n = 267
Text Change: M = 2.81, SD = 0.92, n = 267
Font Change: M = 3.19, SD = 0.96, n = 267

LIKABLE Analysis:
Friedman test: χ² = 98.110, p = 0.000
Significant overall differences detected
Pairwise comparisons:
  Original vs Background: p = 0.000, effec